In [1]:
from PIL import Image, ImageDraw
import numpy as np
import numpy.random as random
import math
import os
import matplotlib.pyplot as plt


In [2]:
x_resolution = 5120
y_resolution = 1440

#radius of seed circle
R = min([x_resolution,y_resolution])//4
print(R)

360


In [3]:
rgbArray = np.zeros((y_resolution,x_resolution,3), 'uint8')


#different background colours
r = (np.ones([y_resolution,x_resolution])*25)
g = (np.ones([y_resolution,x_resolution])*51)
b = (np.ones([y_resolution,x_resolution])*56)

# r = (np.ones([y_resolution,x_resolution])*16)
# g = (np.ones([y_resolution,x_resolution])*37)
# b = (np.ones([y_resolution,x_resolution])*38)

# r = (np.ones([y_resolution,x_resolution])*48)
# g = (np.ones([y_resolution,x_resolution])*10)
# b = (np.ones([y_resolution,x_resolution])*110)

colour_arrays = {'r':r,'g':g,'b':b}
rgbArray[..., 0] = colour_arrays['r']
rgbArray[..., 1] = colour_arrays['g']
rgbArray[..., 2] = colour_arrays['b']
img = Image.fromarray(rgbArray)
#img.show()

In [4]:
#colour gens
colour_collection = []

#3 preset colour pallets for the agents

colour_collection.append([205,126,89])
colour_collection.append([69,115,115])
colour_collection.append([221,178,71])
colour_collection.append([90,104,104])


# colour_collection.append([240,202,45])
# colour_collection.append([20,174,219])
# colour_collection.append([246,125,84])
# colour_collection.append([209,242,149])


# colour_collection.append([11,0,207])
# colour_collection.append([255,45,43])
# colour_collection.append([193,10,40])
#colour_collection.append([115,186,211])


In [5]:
#generate circle coords
origins = []
no_of_origins = 200
for i in range(no_of_origins):
    rnd = i/no_of_origins 
    r = R 
    theta = rnd * (2 * math.pi)
    x = int(x_resolution//2 + r * math.cos(theta))
    y = int(y_resolution//2 + r * math.sin(theta))
    
    origins.append([y,x])
    
    

In [6]:
rgbArray[..., 0] = colour_arrays['r']
rgbArray[..., 1] = colour_arrays['g']
rgbArray[..., 2] = colour_arrays['b']
img = Image.fromarray(rgbArray)
img.show()

In [7]:
#how to change the position of the agent based on its direction
#0 is north, 1 is north east, etc
direction_instructions = {0:(-1,0),1:(-1,1),2:(0,1),3:(1,1),4:(1,0),5:(1,-1),6:(0,-1),7:(-1,-1)}
#double time
#direction_instructions = {0:(-2,0),1:(-2,2),2:(0,2),3:(2,2),4:(2,0),5:(2,-2),6:(0,-2),7:(-2,-2)}

#which new directions are tried when colliding
redirection_choices = {0:(3,4,5),1:(4,5,6),2:(5,6,7),3:(6,7,0),4:(7,0,1),5:(0,1,2),6:(1,2,3),7:(2,3,4)}

In [8]:
taken_cells_log = np.zeros([y_resolution,x_resolution])

In [9]:
class Agent():
    
    def __init__(self,start_position,colour):
        self.position = start_position
        self.colour = colour
        taken_cells_log[self.position[0],self.position[1]] = 1
        self.direction = random.randint(8)
        self.finished = False
        
        #brightness is used for the static B&W propogation map produced at the end. Makes nice wallpapers
        self.brightness_red_step = 255/time
        self.brightness_reduction = 0
        
    def update_position(self):
        
        if self.finished != True:
        
            #initial update
            self.position[0] = self.position[0]+direction_instructions[self.direction][0]
            self.position[1] = self.position[1]+direction_instructions[self.direction][1]

            #check if it hits walls and if the new position is already taken in the taken_cells_log:
            if (y_resolution-2< self.position[0]) or self.position[0]<2  or (x_resolution-2< self.position[1]) or self.position[1]<2 or taken_cells_log[self.position[0],self.position[1]] !=0:

                
                #undo initial change in position
                self.position[0] = self.position[0]-direction_instructions[self.direction][0]
                self.position[1] = self.position[1]-direction_instructions[self.direction][1]

                #get new direction
                new_direction = random.choice(redirection_choices[self.direction])

                self.position[0] = self.position[0]+direction_instructions[new_direction][0]
                self.position[1] = self.position[1]+direction_instructions[new_direction][1]

                self.direction = new_direction
                
            #random chance to change direction
            elif random.random() < 0.0035:
                
                self.position[0] = self.position[0]-direction_instructions[self.direction][0]
                self.position[1] = self.position[1]-direction_instructions[self.direction][1]

                new_direction = random.choice(8)

                self.position[0] = self.position[0]+direction_instructions[new_direction][0]
                self.position[1] = self.position[1]+direction_instructions[new_direction][1]

                self.direction = new_direction
                


            try:
                #add new position to taken_cells_log. Could play around with adding an area around the agent to mean that lines stay away from eachother but difficult for diagonals
                taken_cells_log[self.position[0]:self.position[0]+1,self.position[1]:self.position[1]+1] = int(255 - self.brightness_reduction) #the number stored is 
                self.brightness_reduction += self.brightness_red_step
            
            except IndexError:
                print('position out of bounds')
                print(self.position)
                
            if y_resolution-2< self.position[0] or self.position[0]<2  or (x_resolution-2< self.position[1]) or self.position[1]<2:
                #if new direction is also at the edge (usually when its run into a corner and is stuck)
                #break propogation
                self.finished = True
                
            else:

                for colour in colour_arrays.keys():
                    colour_arrays[colour][self.position[0]-1:self.position[0]+2,self.position[1]-1:self.position[1]+2] = self.colour[colour] #colours a 2x2 around the agent for thicker lines
                    #colour_arrays[colour][self.position[0],self.position[1]] = self.colour[colour] #uncomment for thinner, single pixel lines (good for lower res)
                
        
        

In [10]:
def generate_colour():
    colour = random.randint(len(colour_collection))
    r = colour_collection[colour][0]
    g = colour_collection[colour][1]
    b = colour_collection[colour][2]
    colour_arrays = {'r':r,'g':g,'b':b}
    return colour_arrays

In [11]:

time = 1750 #timesteps rendered
#expect to take around 4 minutes
agents = []

for i in range(no_of_origins):
    #generate list of agents
    agents.append(Agent(origins[i],generate_colour()))

for i in range(time):
    
    if i%100 == 0:
        print('progress:',int(i/time*100),'%')
        
      
    for agent in agents:
        agent.update_position()
    
    
    #Create image for timestep and save
    rgbArray[..., 0] = colour_arrays['r']
    rgbArray[..., 1] = colour_arrays['g']
    rgbArray[..., 2] = colour_arrays['b']
    img = Image.fromarray(rgbArray)
    img.save('png_output/img_'+str(i)+'.png')
    if i % 500 == 0 or i == 100:
        img.show()
        
    

progress: 0 %
progress: 5 %
progress: 11 %
progress: 17 %
progress: 22 %
progress: 28 %
progress: 34 %
progress: 40 %
progress: 45 %
progress: 51 %
progress: 57 %
progress: 62 %
progress: 68 %
progress: 74 %
progress: 80 %
progress: 85 %
progress: 91 %
progress: 97 %


In [ ]:
#I used the video sequencer in blender to create the final video files. highly reccomend. Google "how to make video from image sequence blender"

In [12]:
#create B&W static propagation map
img = Image.fromarray(taken_cells_log)
img.show()